In [1]:
import json
import pandas as pd
import os
from table import *
from table_ast import *
from synthesizer import *
from tabulate import tabulate
from table_cell import *

# small parameter config for tests
test_config = {
                "operators": ["group_sum", "mutate_arithmetic", "group_mutate"],
                "filer_op": ["=="],
                "constants": [3000],
                "aggr_func": ["mean", "sum", "count", "max"],
                "mutate_func": ["mean", "sum", "max", "cumsum"],
                "mutate_op": ["sum"],
                "mutate_function": ["lambda x, y: x - y",
                                    "lambda x, y: x + y",
                                    "lambda x, y: x * y",
                                    "lambda x, y: x / y",
                                    "lambda x: x - (x * 0.1)",
                                    "lambda x: 1"]
            }

def test_run(filename):
    with open(filename, 'r') as filehandler:
        data = json.load(filehandler)
        # join with arithmetic
        # description:
        inputs = data["input_data"]
        # output = data["output_data"]

        if "exp_out" in data.keys():
            p = dict_to_program(data["exp_out"])
            print("=====target annotated output=====")
            print(p.eval(inputs).to_dataframe())
            annotated_output = p.eval(inputs)
        else:
            # annotated_output = load_from_dict(output)
            print("load error")
        candidates = []
        for i in range(1, 4):
            candidates = Synthesizer(test_config).enumerative_synthesis(inputs, annotated_output, i)
            if len(candidates) > 0:
                break
        for p in candidates:
            # print(alignment_result)
            print(p.stmt_string())
            print(tabulate(p.eval(inputs).extract_values(), headers='keys', tablefmt='psql'))
            print(tabulate(p.eval(inputs).extract_traces(), headers='keys', tablefmt='psql'))
            print()
        print(f"number of programs: {len(candidates)}")

In [2]:
test_run("testbenches/001.json")

=====target annotated output=====
                                                                              depno  \
0                                   <10, [ArgOr[(0, 1, 6), (0, 1, 8), (0, 1, 13)]]>   
1            <20, [ArgOr[(0, 1, 0), (0, 1, 3), (0, 1, 7), (0, 1, 10), (0, 1, 12)]]>   
2  <30, [ArgOr[(0, 1, 1), (0, 1, 2), (0, 1, 4), (0, 1, 5), (0, 1, 9), (0, 1, 11)]]>   

                                                                                   mean_sal  
0                                   <2916.67, ('mean', [(0, 2, 6), (0, 2, 8), (0, 2, 13)])>  
1             <2175.0, ('mean', [(0, 2, 0), (0, 2, 3), (0, 2, 7), (0, 2, 10), (0, 2, 12)])>  
2  <1566.67, ('mean', [(0, 2, 1), (0, 2, 2), (0, 2, 4), (0, 2, 5), (0, 2, 9), (0, 2, 11)])>  
t0 <- table_ref(0)
run checker_function!
t0 <- table_ref(0); t1 <- group_sum(t0, ?, ?, ?)
t0 <- table_ref(0); t1 <- group_sum(t0, (1,), ?, ?)
cell trace check
computation check
t0 <- table_ref(0); t1 <- group_sum(t0, (2,), ?, ?)
cell trace c

In [3]:
test_run("testbenches/002.json")

=====target annotated output=====
                   empno               depno                   sal  \
0    <7369, [(0, 0, 0)]>   <20, [(0, 1, 0)]>    <800, [(0, 2, 0)]>   
1    <7499, [(0, 0, 1)]>   <30, [(0, 1, 1)]>   <1600, [(0, 2, 1)]>   
2    <7521, [(0, 0, 2)]>   <30, [(0, 1, 2)]>   <1250, [(0, 2, 2)]>   
3    <7566, [(0, 0, 3)]>   <20, [(0, 1, 3)]>   <2975, [(0, 2, 3)]>   
4    <7654, [(0, 0, 4)]>   <30, [(0, 1, 4)]>   <1250, [(0, 2, 4)]>   
5    <7698, [(0, 0, 5)]>   <30, [(0, 1, 5)]>   <2850, [(0, 2, 5)]>   
6    <7782, [(0, 0, 6)]>   <10, [(0, 1, 6)]>   <2450, [(0, 2, 6)]>   
7    <7788, [(0, 0, 7)]>   <20, [(0, 1, 7)]>   <3000, [(0, 2, 7)]>   
8    <7839, [(0, 0, 8)]>   <10, [(0, 1, 8)]>   <5000, [(0, 2, 8)]>   
9    <7844, [(0, 0, 9)]>   <30, [(0, 1, 9)]>   <1500, [(0, 2, 9)]>   
10  <7876, [(0, 0, 10)]>  <20, [(0, 1, 10)]>  <1100, [(0, 2, 10)]>   
11  <7900, [(0, 0, 11)]>  <30, [(0, 1, 11)]>   <950, [(0, 2, 11)]>   
12  <7902, [(0, 0, 12)]>  <20, [(0, 1, 12)]>  <3000, [(0

In [4]:
test_run("testbenches/004.json")

=====target annotated output=====
                 id                  total  \
0  <1, [(0, 0, 0)]>   <24.78, [(0, 1, 0)]>   
1  <2, [(0, 0, 1)]>   <28.54, [(0, 1, 1)]>   
2  <3, [(0, 0, 2)]>   <48.69, [(0, 1, 2)]>   
3  <4, [(0, 0, 3)]>  <-16.39, [(0, 1, 3)]>   
4  <5, [(0, 0, 4)]>   <29.92, [(0, 1, 4)]>   
5  <6, [(0, 0, 5)]>    <12.5, [(0, 1, 5)]>   
6  <7, [(0, 0, 6)]>    <10.2, [(0, 1, 6)]>   
7  <8, [(0, 0, 7)]>    <5.22, [(0, 1, 7)]>   

                                                                                                           COL_0  \
0  <143.46, [('sum', [(0, 1, 0), (0, 1, 1), (0, 1, 2), (0, 1, 3), (0, 1, 4), (0, 1, 5), (0, 1, 6), (0, 1, 7)])]>   
1  <143.46, [('sum', [(0, 1, 0), (0, 1, 1), (0, 1, 2), (0, 1, 3), (0, 1, 4), (0, 1, 5), (0, 1, 6), (0, 1, 7)])]>   
2  <143.46, [('sum', [(0, 1, 0), (0, 1, 1), (0, 1, 2), (0, 1, 3), (0, 1, 4), (0, 1, 5), (0, 1, 6), (0, 1, 7)])]>   
3  <143.46, [('sum', [(0, 1, 0), (0, 1, 1), (0, 1, 2), (0, 1, 3), (0, 1, 4), (0, 1, 5),

In [5]:
test_run("testbenches/005.json")

=====target annotated output=====
                                                                date  \
0  <2015-09-24, [ArgOr[(0, 1, 0), (0, 1, 1), (0, 1, 2), (0, 1, 3)]]>   
1             <2015-09-30, [ArgOr[(0, 1, 4), (0, 1, 5), (0, 1, 6)]]>   
2                                   <2015-11-16, [ArgOr[(0, 1, 7)]]>   

                                                    count_trip_id  \
0  <4, [('count', [(0, 0, 0), (0, 0, 1), (0, 0, 2), (0, 0, 3)])]>   
1             <3, [('count', [(0, 0, 4), (0, 0, 5), (0, 0, 6)])]>   
2                                   <1, [('count', [(0, 0, 7)])]>   

                                                                                                                                              COL_0  
0                                                                        <4, ('cumsum', [('count', [(0, 0, 0), (0, 0, 1), (0, 0, 2), (0, 0, 3)])])>  
1                          <7, ('cumsum', [('count', [(0, 0, 0), (0, 0, 1), (0, 0, 2), (0, 0, 3)]), 

In [6]:
test_run("testbenches/007.json")

=====target annotated output=====
                amount         circle_id  \
0  <1000, [(0, 0, 0)]>  <1, [(0, 1, 0)]>   
1  <3000, [(0, 0, 1)]>  <2, [(0, 1, 1)]>   
2  <2000, [(0, 0, 2)]>  <3, [(0, 1, 2)]>   
3  <3000, [(0, 0, 3)]>  <1, [(0, 1, 3)]>   
4  <2500, [(0, 0, 4)]>  <2, [(0, 1, 4)]>   
5  <3750, [(0, 0, 5)]>  <3, [(0, 1, 5)]>   
6  <2000, [(0, 0, 6)]>  <1, [(0, 1, 6)]>   
7  <2500, [(0, 0, 7)]>  <2, [(0, 1, 7)]>   
8  <1450, [(0, 0, 8)]>  <3, [(0, 1, 8)]>   

                                                                                                                      COL_0  
0                                                                                           <1000, ('cumsum', [(0, 0, 0)])>  
1                                                                                <4000, ('cumsum', [(0, 0, 0), (0, 0, 1)])>  
2                                                                     <6000, ('cumsum', [(0, 0, 0), (0, 0, 1), (0, 0, 2)])>  
3                    

In [7]:
test_run("testbenches/008.json")

=====target annotated output=====
           cust_country             grade       outstanding_amt  \
0     <UK, [(0, 0, 0)]>  <2, [(0, 1, 0)]>   <4000, [(0, 2, 0)]>   
1    <USA, [(0, 0, 1)]>  <2, [(0, 1, 1)]>   <6000, [(0, 2, 1)]>   
2    <USA, [(0, 0, 2)]>  <3, [(0, 1, 2)]>   <6000, [(0, 2, 2)]>   
3  <India, [(0, 0, 3)]>  <2, [(0, 1, 3)]>   <8000, [(0, 2, 3)]>   
4     <UK, [(0, 0, 4)]>  <2, [(0, 1, 4)]>   <6000, [(0, 2, 4)]>   
5     <UK, [(0, 0, 5)]>  <1, [(0, 1, 5)]>  <11000, [(0, 2, 5)]>   
6    <USA, [(0, 0, 6)]>  <3, [(0, 1, 6)]>   <3000, [(0, 2, 6)]>   

                                                COL_0  
0  <3600.0, ('lambda x: x - (x * 0.1)', [(0, 2, 0)])>  
1  <5400.0, ('lambda x: x - (x * 0.1)', [(0, 2, 1)])>  
2  <5400.0, ('lambda x: x - (x * 0.1)', [(0, 2, 2)])>  
3  <7200.0, ('lambda x: x - (x * 0.1)', [(0, 2, 3)])>  
4  <5400.0, ('lambda x: x - (x * 0.1)', [(0, 2, 4)])>  
5  <9900.0, ('lambda x: x - (x * 0.1)', [(0, 2, 5)])>  
6  <2700.0, ('lambda x: x - (x * 0.1)

In [8]:
test_run("testbenches/010.json")

=====target annotated output=====
                        state                count  \
0      <Alabama, [(0, 0, 0)]>  <1667, [(0, 1, 0)]>   
1       <Alaska, [(0, 0, 1)]>   <507, [(0, 1, 1)]>   
2      <Alabama, [(0, 0, 2)]>   <930, [(0, 1, 2)]>   
3      <Arizona, [(0, 0, 3)]>  <1352, [(0, 1, 3)]>   
4   <California, [(0, 0, 4)]>  <1817, [(0, 1, 4)]>   
5   <California, [(0, 0, 5)]>  <2302, [(0, 1, 5)]>   
6  <Connecticut, [(0, 0, 6)]>  <1488, [(0, 1, 6)]>   

                                       COL_0  \
0  <2597, [('sum', [(0, 1, 0), (0, 1, 2)])]>   
1              <507, [('sum', [(0, 1, 1)])]>   
2  <2597, [('sum', [(0, 1, 0), (0, 1, 2)])]>   
3             <1352, [('sum', [(0, 1, 3)])]>   
4  <4119, [('sum', [(0, 1, 4), (0, 1, 5)])]>   
5  <4119, [('sum', [(0, 1, 4), (0, 1, 5)])]>   
6             <1488, [('sum', [(0, 1, 6)])]>   

                                                                                               COL_1  \
0  <10063, [('sum', [(0, 1, 0), (0, 1, 1), (

In [9]:
test_run("testbenches/011.json")

=====target annotated output=====
                  X                 Y                 Z  \
0  <1, [(0, 0, 0)]>  <2, [(0, 1, 0)]>  <3, [(0, 2, 0)]>   
1  <2, [(0, 0, 1)]>  <3, [(0, 1, 1)]>  <4, [(0, 2, 1)]>   
2  <3, [(0, 0, 2)]>  <4, [(0, 1, 2)]>  <5, [(0, 2, 2)]>   

                                                   COL_0  \
0  <3, [('lambda x, y: x + y', [(0, 0, 0), (0, 1, 0)])]>   
1  <5, [('lambda x, y: x + y', [(0, 0, 1), (0, 1, 1)])]>   
2  <7, [('lambda x, y: x + y', [(0, 0, 2), (0, 1, 2)])]>   

                                                                                       COL_1  
0   <6, ('lambda x, y: x + y', [(0, 2, 0), ('lambda x, y: x + y', [(0, 0, 0), (0, 1, 0)])])>  
1   <9, ('lambda x, y: x + y', [(0, 2, 1), ('lambda x, y: x + y', [(0, 0, 1), (0, 1, 1)])])>  
2  <12, ('lambda x, y: x + y', [(0, 2, 2), ('lambda x, y: x + y', [(0, 0, 2), (0, 1, 2)])])>  
t0 <- table_ref(0)
run checker_function!
t0 <- table_ref(0); t1 <- group_sum(t0, ?, ?, ?)
pruned by group ke

In [10]:
test_run("testbenches/012.json")

=====target annotated output=====
            Net Cash Flow                 Month  \
0    <80000, [(0, 0, 0)]>  <Start, [(0, 1, 0)]>   
1    <-5003, [(0, 0, 1)]>    <Apr, [(0, 1, 1)]>   
2   <-16700, [(0, 0, 2)]>    <May, [(0, 1, 2)]>   
3    <48802, [(0, 0, 3)]>    <Jun, [(0, 1, 3)]>   
4   <-11198, [(0, 0, 4)]>    <Jul, [(0, 1, 4)]>   
5   <-35260, [(0, 0, 5)]>    <Aug, [(0, 1, 5)]>   
6    <18220, [(0, 0, 6)]>    <Sep, [(0, 1, 6)]>   
7   <-23840, [(0, 0, 7)]>    <Oct, [(0, 1, 7)]>   
8    <43250, [(0, 0, 8)]>    <Nov, [(0, 1, 8)]>   
9   <-18280, [(0, 0, 9)]>    <Dec, [(0, 1, 9)]>   
10  <26670, [(0, 0, 10)]>   <Jan, [(0, 1, 10)]>   
11  <15000, [(0, 0, 11)]>   <Feb, [(0, 1, 11)]>   
12  <24750, [(0, 0, 12)]>   <Mar, [(0, 1, 12)]>   

                                                                                                                                                                         COL_0  \
0                                                                        

In [11]:
test_run("testbenches/013.json")

=====target annotated output=====
              Value             alpha              beta             gamma  \
0  <A, [(0, 0, 0)]>  <2, [(0, 1, 0)]>  <2, [(0, 2, 0)]>  <3, [(0, 3, 0)]>   
1  <B, [(0, 0, 1)]>  <2, [(0, 1, 1)]>  <3, [(0, 2, 1)]>  <3, [(0, 3, 1)]>   
2  <C, [(0, 0, 2)]>  <3, [(0, 1, 2)]>  <3, [(0, 2, 2)]>  <3, [(0, 3, 2)]>   
3  <D, [(0, 0, 3)]>  <3, [(0, 1, 3)]>  <4, [(0, 2, 3)]>  <2, [(0, 3, 3)]>   
4  <E, [(0, 0, 4)]>  <4, [(0, 1, 4)]>  <3, [(0, 2, 4)]>  <2, [(0, 3, 4)]>   

             Totals                         COL_0  
0  <7, [(0, 4, 0)]>  <7, ('cumsum', [(0, 4, 0)])>  
1  <8, [(0, 4, 1)]>  <8, ('cumsum', [(0, 4, 1)])>  
2  <9, [(0, 4, 2)]>  <9, ('cumsum', [(0, 4, 2)])>  
3  <9, [(0, 4, 3)]>  <9, ('cumsum', [(0, 4, 3)])>  
4  <9, [(0, 4, 4)]>  <9, ('cumsum', [(0, 4, 4)])>  
t0 <- table_ref(0)
run checker_function!
t0 <- table_ref(0); t1 <- group_sum(t0, ?, ?, ?)
t0 <- table_ref(0); t1 <- group_sum(t0, (1,), ?, ?)
cell trace check
=====Cell Check Result=====
chec

In [13]:
test_run("testbenches/014.json")

=====target annotated output=====
                                                                                                                                                                                                                                                                                                            Product  \
0                                                                                                                                                                                                                                                 <Apple, [ArgOr[(0, 1, 31), (0, 1, 35), (0, 1, 88), (0, 1, 200)]]>   
1                                                                                                                                                                                                                        <Apple, [ArgOr[(0, 1, 9), (0, 1, 53), (0, 1, 63), (0, 1, 107), (0, 1, 143), (0, 1, 204)]]>   
2                                

In [14]:
test_run("testbenches/015.json")

=====target annotated output=====
                     Complaint Type               Count  \
0          <Too noisy, [(0, 0, 0)]>   <27, [(0, 1, 0)]>   
1         <Overpriced, [(0, 0, 1)]>  <789, [(0, 1, 1)]>   
2  <Food is tasteless, [(0, 0, 2)]>   <65, [(0, 1, 2)]>   
3     <Food not fresh, [(0, 0, 3)]>    <9, [(0, 1, 3)]>   
4  <Food is too salty, [(0, 0, 4)]>   <15, [(0, 1, 4)]>   
5          <Not clean, [(0, 0, 5)]>   <30, [(0, 1, 5)]>   
6   <Unfriendly staff, [(0, 0, 6)]>   <12, [(0, 1, 6)]>   
7          <Wait time, [(0, 0, 7)]>  <109, [(0, 1, 7)]>   
8      <No atmosphere, [(0, 0, 8)]>   <45, [(0, 1, 8)]>   
9     <Small portions, [(0, 0, 9)]>  <621, [(0, 1, 9)]>   

                                                                                                                                COL_0  
0                                                                                                       <27, ('cumsum', [(0, 1, 0)])>  
1                                           

In [ ]:
test_run("testbenches/016.json")

=====target annotated output=====
                                                                                       data_ora  \
0      <06_00, [ArgOr[(0, 0, 0), (0, 0, 1), (0, 0, 96), (0, 0, 97), (0, 0, 192), (0, 0, 193)]]>   
1      <06_00, [ArgOr[(0, 0, 2), (0, 0, 3), (0, 0, 98), (0, 0, 99), (0, 0, 194), (0, 0, 195)]]>   
2    <06_01, [ArgOr[(0, 0, 4), (0, 0, 5), (0, 0, 100), (0, 0, 101), (0, 0, 196), (0, 0, 197)]]>   
3    <06_01, [ArgOr[(0, 0, 6), (0, 0, 7), (0, 0, 102), (0, 0, 103), (0, 0, 198), (0, 0, 199)]]>   
4    <06_02, [ArgOr[(0, 0, 8), (0, 0, 9), (0, 0, 104), (0, 0, 105), (0, 0, 200), (0, 0, 201)]]>   
..                                                                                          ...   
892                                                <30_14, [ArgOr[(0, 0, 1976), (0, 0, 1977)]]>   
893                                                <30_14, [ArgOr[(0, 0, 1978), (0, 0, 1979)]]>   
894                                                <30_15, [ArgOr[(0, 0, 19

In [18]:
test_run("testbenches/017.json")

=====target annotated output=====
                    id              Value            Win / Loss  \
0     <0, [(0, 0, 0)]>  <10, [(0, 1, 0)]>    <Win, [(0, 2, 0)]>   
1     <1, [(0, 0, 1)]>   <2, [(0, 1, 1)]>   <Loss, [(0, 2, 1)]>   
2     <2, [(0, 0, 2)]>  <12, [(0, 1, 2)]>    <Win, [(0, 2, 2)]>   
3     <3, [(0, 0, 3)]>   <4, [(0, 1, 3)]>   <Loss, [(0, 2, 3)]>   
4     <4, [(0, 0, 4)]>   <5, [(0, 1, 4)]>    <Win, [(0, 2, 4)]>   
5     <5, [(0, 0, 5)]>  <16, [(0, 1, 5)]>   <Loss, [(0, 2, 5)]>   
6     <6, [(0, 0, 6)]>   <9, [(0, 1, 6)]>    <Win, [(0, 2, 6)]>   
7     <7, [(0, 0, 7)]>  <18, [(0, 1, 7)]>   <Loss, [(0, 2, 7)]>   
8     <8, [(0, 0, 8)]>  <12, [(0, 1, 8)]>    <Win, [(0, 2, 8)]>   
9     <9, [(0, 0, 9)]>   <3, [(0, 1, 9)]>   <Loss, [(0, 2, 9)]>   
10  <10, [(0, 0, 10)]>  <4, [(0, 1, 10)]>   <Win, [(0, 2, 10)]>   
11  <11, [(0, 0, 11)]>  <4, [(0, 1, 11)]>  <Loss, [(0, 2, 11)]>   
12  <12, [(0, 0, 12)]>  <1, [(0, 1, 12)]>   <Win, [(0, 2, 12)]>   

                           

In [2]:
test_run("testbenches/018.json")

=====target annotated output=====
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      cut  \
0                                                                                                                                                                                                                                                                                                                                                                                                                        

In [4]:
test_run("testbenches/019.json")

=====target annotated output=====
               obs_num                set                  x_  \
0     <1, [(0, 0, 0)]>   <1, [(0, 1, 0)]>   <10, [(0, 2, 0)]>   
1     <1, [(0, 0, 1)]>   <2, [(0, 1, 1)]>   <10, [(0, 2, 1)]>   
2     <1, [(0, 0, 2)]>   <3, [(0, 1, 2)]>   <10, [(0, 2, 2)]>   
3     <1, [(0, 0, 3)]>   <4, [(0, 1, 3)]>    <8, [(0, 2, 3)]>   
4     <2, [(0, 0, 4)]>   <1, [(0, 1, 4)]>    <8, [(0, 2, 4)]>   
5     <2, [(0, 0, 5)]>   <2, [(0, 1, 5)]>    <8, [(0, 2, 5)]>   
6     <2, [(0, 0, 6)]>   <3, [(0, 1, 6)]>    <8, [(0, 2, 6)]>   
7     <2, [(0, 0, 7)]>   <4, [(0, 1, 7)]>    <8, [(0, 2, 7)]>   
8     <3, [(0, 0, 8)]>   <1, [(0, 1, 8)]>   <13, [(0, 2, 8)]>   
9     <3, [(0, 0, 9)]>   <2, [(0, 1, 9)]>   <13, [(0, 2, 9)]>   
10   <3, [(0, 0, 10)]>  <3, [(0, 1, 10)]>  <13, [(0, 2, 10)]>   
11   <3, [(0, 0, 11)]>  <4, [(0, 1, 11)]>   <8, [(0, 2, 11)]>   
12   <4, [(0, 0, 12)]>  <1, [(0, 1, 12)]>   <9, [(0, 2, 12)]>   
13   <4, [(0, 0, 13)]>  <2, [(0, 1, 13)]>   <9, [(0, 2, 

In [5]:
test_run("testbenches/020.json")

=====target annotated output=====
                       fac                 score  \
0    <trial1, [(0, 0, 0)]>   <1.04, [(0, 1, 0)]>   
1    <trial1, [(0, 0, 1)]>    <1.5, [(0, 1, 1)]>   
2    <trial1, [(0, 0, 2)]>   <2.18, [(0, 1, 2)]>   
3    <trial1, [(0, 0, 3)]>   <1.01, [(0, 1, 3)]>   
4    <trial1, [(0, 0, 4)]>   <1.84, [(0, 1, 4)]>   
5    <trial1, [(0, 0, 5)]>   <1.35, [(0, 1, 5)]>   
6    <trial1, [(0, 0, 6)]>   <0.91, [(0, 1, 6)]>   
7    <trial1, [(0, 0, 7)]>   <1.69, [(0, 1, 7)]>   
8    <trial1, [(0, 0, 8)]>   <3.09, [(0, 1, 8)]>   
9    <trial1, [(0, 0, 9)]>   <3.25, [(0, 1, 9)]>   
10  <trial2, [(0, 0, 10)]>  <1.71, [(0, 1, 10)]>   
11  <trial2, [(0, 0, 11)]>  <2.61, [(0, 1, 11)]>   
12  <trial2, [(0, 0, 12)]>  <4.32, [(0, 1, 12)]>   
13  <trial2, [(0, 0, 13)]>  <2.85, [(0, 1, 13)]>   
14  <trial2, [(0, 0, 14)]>  <3.22, [(0, 1, 14)]>   
15  <trial2, [(0, 0, 15)]>  <4.44, [(0, 1, 15)]>   
16  <trial2, [(0, 0, 16)]>  <2.14, [(0, 1, 16)]>   
17  <trial2, [(0, 0, 17)]>  <2